Import dependencies

In [35]:
import cv2
import os
import numpy as np
import tensorflow as tf 
from tensorflow.keras.layers import Layer           

In [8]:
if ( not (os.path.isdir(os.path.join('application_data', 'verification_images')))):
    os.makedirs(os.path.join('application_data', 'verification_images'))

if ( not (os.path.isdir(os.path.join('application_data', 'input_image')))):
    os.makedirs(os.path.join('application_data', 'input_image'))

preprocess function

In [9]:
def preprocess(file_path):

    byte_img = tf.io.read_file(file_path)
    img = tf.io.decode_jpeg(byte_img)

    img = tf.image.resize(img, (100, 100))
    img = img/ 255.0

    return img

Verification function

In [40]:
def verify( model, detection_threshold, verification_threshold, img):

    results = []
    # input_img = preprocess(os.path.join('application_data', 'input_image')
    input_img = preprocess(img)
    for image in os.listdir(os.path.join('application_data', 'verification_images')):
        validation_img = preprocess(os.path.join('application_data', 'verification_images', image))

        # make predictions
        result = model.predict(list(np.expand_dims([input_img, validation_img], axis=1)))
        results.append(result)

    detection = np.sum(np.array(results) > detection_threshold)
    print('detection score is {}'.format(detection))

    # verification threshold
    verification = detection / len(os.listdir(os.path.join('application_data', 'verification_images'))) 
    print('verification percentage is {}'.format(verification))
    verified = verification > verification_threshold

    return results, verified

Get Input Image for verification

In [46]:
def get_input_image():

    image_folder = os.path.join('application_data', 'input_image')
    input_img = ''

    for filename in os.listdir(image_folder):
        input_img = os.path.join(image_folder, filename)
        break

    return input_img

Load trained siamese model

In [28]:
# Define L1Dist model class

class L1Dist(Layer):
    def __init__(self, **kwargs):
        super().__init__()

    def call(self, input_embedding, validation_embedding):
        return tf.math.abs(input_embedding - validation_embedding)
        

In [29]:
L1Dist

__main__.L1Dist

In [44]:
# the trained siamese model is roughtly 75.2 MB in size
siamese_model = tf.keras.models.load_model('siamesemodelv2.h5',
                                   custom_objects={'L1Dist':L1Dist, 'BinaryCrossentropy':tf.losses.BinaryCrossentropy})


Call verification function to verify image

In [47]:
input_img = get_input_image()
results, verified = verify(siamese_model, 0.5, 0.5,input_img)
print(verified)

1/1 [==============================] - 2s 2s/step
detection score is 0
verification percentage is 0.0
False
